# Generador de personas aleatorias

In [35]:
import math
import random as rm
import numpy as np
import pandas as pd
from faker import Faker
import datetime as dt
import pytz
import re
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
from funciones import quitar_acentos
from typing import Union

In [27]:
def sacar_curp(nombre: str, apellido_paterno: str, apellido_materno: str, sexo: str, fecha_nac: Union[str, dt.datetime], estado_nac: str) -> str:
	nombre = quitar_acentos(nombre).upper().replace('Ñ', 'X')
	apellido_paterno = quitar_acentos(apellido_paterno).upper().replace('Ñ', 'X')
	apellido_materno = quitar_acentos(apellido_materno).upper().replace('Ñ', 'X')
	estado_nac = quitar_acentos(estado_nac.replace(' ', '_').lower())
	if isinstance(fecha_nac, str): fecha_nac = dt.datetime.strptime(fecha_nac, '%Y-%m-%d')
	# Primera letra y vocal del primer apellido
	curp = apellido_paterno[0:2].upper()
	# Primera letra del segundo apellido
	curp += apellido_materno[0:1].upper()
	# Primera letra del nombre de pila
	curp += nombre[0:1].upper()
	# Fecha de nacimiento (2 últimos dígitos del año, 2 del mes y 2 del día de nacimiento)
	curp += fecha_nac.strftime('%y%m%d')
	# Letra del sexo (H o M)
	curp += sexo[0:1].upper()
	# Dos letras correspondientes a la entidad de nacimiento segun RENAPO (Sonora => SR)
	# en el caso de extranjeros, se marca como NE (Nacido Extranjero)
	clave_estados = {
		'aguascalientes': 'AS', 'baja_california': 'BC', 'baja_california_sur': 'BS',
		'campeche': 'CC', 'coahuila': 'CL', 'colima': 'CM', 'chiapas': 'CS', 'chihuahua': 'CH',
		'distrito_federal': 'DF', 'ciudad_de_mexico': 'DF', 'durango': 'DG', 'guanajuato': 'GT',
		'guerrero': 'GR', 'hidalgo': 'HG', 'jalisco': 'JC', 'mexico': 'MC', 'estado_de_mexico': 'MC',
		'michoacan': 'MN', 'morelos': 'MS', 'nayarit': 'NT', 'nuevo_leon': 'NL', 'oaxaca': 'OC',
		'puebla': 'PL', 'queretaro': 'QT', 'quintana_roo': 'QR', 'san_luis_potosi': 'SP', 'sinaloa': 'SL',
		'sonora': 'SR', 'tabasco': 'TC', 'tamaulipas': 'TS', 'tlaxcala': 'TL', 'veracruz': 'VZ',
		'yucatan': 'YN', 'zacatecas': 'ZS'
	}
	if estado_nac in clave_estados:
		curp += clave_estados[estado_nac]
	else: curp += 'NE'
	# Primera consonante interna del primer apellido
	curp += re.sub(f'[aeiou]', '', apellido_paterno)[1:2].upper()
	# Primera consonante interna del segundo apellido
	curp += re.sub(f'[aeiou]', '', apellido_materno)[1:2].upper()
	# Primera consonante interna del nombre
	curp += re.sub(f'[aeiou]', '', nombre)[1:2].upper()
	# Dígito verificador del 0-9 para fechas de nacimiento hasta el año 1999 y A-Z para fechas de nacimiento a partir del 2000
	int_char = [rm.randint(0, 9), chr(rm.randint(65, 90))]
	curp += str(int_char[0] if fecha_nac.year <= 1999 else int_char[1])
	# Homoclave, para evitar duplicaciones
	curp += str(rm.randint(0, 9))
	return curp

geolocator = Nominatim(user_agent = 'geoapiExercises')
geocode = lambda query: geolocator.geocode(query, featuretype = 'city', language = 'es')
reverse = lambda coords: geolocator.reverse(coords, language = 'es')

def obtener_coordenadas(ciudad, estado, pais = 'Mexico') -> dict:
	try:
		# Hacer peticion a un api para obtener coordenadas del centro y extremas
		location = geocode(f'{ciudad} {estado} {pais}')
		# En caso de que no se haya encontrado nada
		if not location: raise BaseException(f'No se encontraron coordenadas de "{ciudad}", "{estado}", "{pais}"')
		location = location.raw
		# Obtener coordenadas del centro
		centro = (float(location['lat']), float(location['lon']))
		# Obtener extremos originales
		minx, maxx, miny, maxy = [float(i) for i in location['boundingbox']]
		ne, no, se, so = [(maxx, maxy), (maxx, miny), (minx, maxy), (minx, miny)]
		print(f'\033[33mobtener_coordenadas()\033[0m: Coordenadas encontradas en "{ciudad}", "{estado}", "{pais}"')
	except BaseException as e:
		print(f'\033[31mobtener_coordenadas()\033[0m:', e)
		return {}
	return {'centro': centro, 'noreste': ne, 'noroeste': no, 'sureste': se, 'suroeste': so}

def generar_puntos_random(coords: dict, n = 1, reducir = True) -> list:
	coords_necesarias = ('centro', 'suroeste', 'noreste')
	if not all(k in coords for k in coords_necesarias):
		print('\033[33mgenerar_puntos_random()\033[0m: No se encontro alguna de las coordenadas necesarias:', coords_necesarias)
		return []
	# Obtener coordenadas del centro
	cenx, ceny = coords['centro']
	# Obtener extremos originales
	minx, miny = coords['suroeste']
	maxx, maxy = coords['noreste']
	# Hallar punto medio entre (suroeste centro) y (centro noreste)
	minx, miny = ((minx + cenx) / 2), ((miny + ceny) / 2)
	maxx, maxy = ((cenx + maxx) / 2), ((ceny + maxy) / 2)
	# True para reducir el area de la ciudad
	if reducir:
		# Hallar punto medio de puntos medios y centro
		minx, miny = ((minx + cenx) / 2), ((miny + ceny) / 2)
		maxx, maxy = ((cenx + maxx) / 2), ((ceny + maxy) / 2)
	# Obtener n puntos aleatorios dentro del area
	puntos_random = []
	while len(puntos_random) < n:
		puntos_random.append((rm.uniform(minx, maxx), rm.uniform(miny, maxy)))
	return puntos_random

def buscar_direccion(puntos_random: list) -> dict:
	dirrr = {}
	listo = False
	for punto_random in puntos_random:
		dirr = {}
		address = reverse(punto_random)
		if not address: continue
		direccion = address.raw['address']
		print(direccion)
		# Ver que campos se encontraron
		# https://nominatim.org/release-docs/latest/api/Output/#addressdetails
		# amenity, building, shop
		if any(k in direccion for k in ('amenity', 'building', 'shop', 'office', 'leisure')):
			if 'amenity' in direccion: dirr['local'] = direccion['amenity']
			elif 'building' in direccion: dirr['local'] = direccion['building']
			elif 'shop' in direccion: dirr['local'] = direccion['shop']
			elif 'office' in direccion: dirr['local'] = direccion['office']
			elif 'leisure' in direccion: dirr['local'] = direccion['leisure']
		# road
		if 'road' in direccion: dirr['calle'] = direccion['road']
		# house_number
		if 'house_number' in direccion: dirr['numero'] = direccion['house_number']
		else: dirr['numero'] = 'S/N'
		# neighbourhood, residential, suburb, quarter
		if any(k in direccion for k in ('neighbourhood', 'residential', 'suburb', 'quarter')):
			if 'neighbourhood' in direccion: dirr['colonia'] = direccion['neighbourhood']
			elif 'residential' in direccion: dirr['colonia'] = direccion['residential']
			elif 'suburb' in direccion: dirr['colonia'] = direccion['suburb']
			elif 'quarter' in direccion: dirr['colonia'] = direccion['quarter']
		# postcode
		if 'postcode' in direccion: dirr['cp'] = direccion['postcode']
		# city, town, village, hamlet
		if any(k in direccion for k in ('city', 'town', 'village', 'hamlet')):
			if 'city' in direccion: dirr['ciudad'] = direccion['city']
			elif 'town' in direccion: dirr['ciudad'] = direccion['town']
			elif 'village' in direccion: dirr['ciudad'] = direccion['village']
			elif 'hamlet' in direccion: dirr['ciudad'] = direccion['hamlet']
		# county
		if 'county' in direccion: dirr['municipio'] = direccion['county']
		# state
		if 'state' in direccion: dirr['estado'] = direccion['state']
		# country
		if 'country' in direccion: dirr['pais'] = direccion['country']
		# Si se encontro al menos calle y cp
		if all(k in dirr for k in ('calle', 'numero')): listo = True
		# Si ya se encontro una buena direccion, dejar de buscar
		if listo:
			print('\033[33mbuscar_direccion()\033[0m: Direccion encontrada en', punto_random)
			print('\033[32m', dirr, '\033[m')
			dirrr = dirr
			break
	if not listo: print('\033[33mbuscar_direccion()\033[0m: No se encontro una direccion precisa')
	return dirrr

In [56]:
# grados a radianes
def deg2rad(degrees):
	return math.pi * degrees / 180.0

# radianes a grados
def rad2deg(radians):
	return 180.0 * radians / math.pi

# Calcular radio de la tierra (R) en metros con una latitud dada segun WGS-84
def radio_terrestre_wgs84(lat):
	# http://en.wikipedia.org/wiki/Earth_radius
	# Semi-ejes de WGS-84
	semieje_mayor_a = 6378137.0  # Semieje mayor a [metros]
	semieje_menor_b = 6356752.31424  # Semieje menor b [metros]

	an = (semieje_mayor_a ** 2) * math.cos(lat)
	bn = (semieje_menor_b ** 2) * math.sin(lat)
	ad = semieje_mayor_a * math.cos(lat)
	bd = semieje_menor_b * math.sin(lat)
	return math.sqrt((an ** 2 + bn ** 2) / (ad ** 2 + bd ** 2))

# Bounding box surrounding the point at given coordinates,
# assuming local approximation of Earth surface as a sphere
# of radius given by WGS84
def bounding_box(coordenadas: tuple, radio_en_km = 10):
	# convertir latitud y longitud a radianes
	lat_rad = deg2rad(coordenadas[0])
	lon_rad = deg2rad(coordenadas[1])
	# ampliar el radio de busqueda
	radio = 1000 * radio_en_km

	# Radio terrestre en una latitud dada
	radius = radio_terrestre_wgs84(lat_rad)
	# Radius of the parallel at given latitude
	pradius = radius * math.cos(lat_rad)

	# Calcular limites y convertirlos a grados
	lat_min = rad2deg(lat_rad - radio / radius)
	lon_min = rad2deg(lon_rad - radio / pradius)
	lat_max = rad2deg(lat_rad + radio / radius)
	lon_max = rad2deg(lon_rad + radio / pradius)
	return [(lat_min, lon_min), (lat_max, lon_max)]

coords = obtener_coordenadas('guadalajara', 'jalisco')
print(coords['centro'])
bounding_box(coords['centro'])

obtener_coordenadas(): Coordenadas encontradas en "guadalajara", "jalisco", "Mexico"
(20.6720375, -103.338396)


[(20.582168696461558, -103.43444904140036),
 (20.761906303538442, -103.24234295859965)]

In [29]:
reap = pd.read_csv('./conjunto_datos/relacion_edad_altura_peso.csv')
nombres = pd.read_csv('./catalogos/nombres.csv')
profesiones = pd.read_csv('./catalogos/profesiones.csv', header = None)
file_ciudades = './catalogos/ciudades.csv'
ct_ciudades = pd.read_csv(file_ciudades)

def personas_aleatorias(n = 5):
	fake = Faker(['es_MX'])
	personas = []
	for i in range(n):
		print(f'\033[34mIdx{i}\033[0m')
		sexo = rm.choice(['H', 'M'])
		# generar tipo de sangre y asignar probabilidad a cada valor para que salga
		tipo_sangre = rm.choices(['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-'], weights = [30, 6, 9, 2, 4, 1, 39, 9])
		tipo_sangre = tipo_sangre[0]

		fecha_nac = fake.date_of_birth(minimum_age = int(reap['edad'].min()))
		# calcular edad en base a la fecha de nacimiento
		dia_actual = dt.date.today()
		edad = dia_actual.year - fecha_nac.year - ((dia_actual.month, dia_actual.day) < (fecha_nac.month, fecha_nac.day))

		# dependiendo de la edad se busca el margen de alturas y pesos que tenga esa edad
		# si es mayor que la edad maxima que este registrada se le da el margen de esa edad maxima
		# mucha gente deja de crecer a los 20
		if edad <= reap['edad'].max(): criter = (reap['edad'] == edad)
		else: criter = (reap['edad'] == reap['edad'].max())
		margen_ap = reap[criter].to_dict(orient = 'records')[0]

		# dependiendo del sexo se generan nombres femeninos o masculinos
		# y estaturas y pesos dentro de su margen correspondiente
		if sexo == 'H':
			titulo = rm.choice(['Sr.', 'Dr.', 'Mtro.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['h'])
			altura = round(rm.uniform(margen_ap['lim_inf.altura.h.cm'], margen_ap['lim_sup.altura.h.cm']), 2)
			peso = round(rm.uniform(margen_ap['lim_inf.peso.h.kg'], margen_ap['lim_sup.peso.h.kg']), 2)
		else:
			titulo = rm.choice(['Sra.', 'Dra.', 'Mtra.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['m'])
			altura = round(rm.uniform(margen_ap['lim_inf.altura.m.cm'], margen_ap['lim_sup.altura.m.cm']), 2)
			peso = round(rm.uniform(margen_ap['lim_inf.peso.m.kg'], margen_ap['lim_sup.peso.m.kg']), 2)

		apellido_paterno = fake.last_name()
		apellido_materno = fake.last_name()

		id_estado = rm.choice(ct_ciudades['id_estado'].unique())
		ct_ciudad = ct_ciudades[ct_ciudades['id_estado'] == id_estado].sample()
		ct_ciudad_index_row = ct_ciudad.index.values[0]
		ct_ciudad = ct_ciudad.to_dict(orient = 'records')[0]

		ciudad = ct_ciudad['ciudad']
		estado = ct_ciudad['estado']
		curp = sacar_curp(nombre, apellido_paterno, apellido_materno, sexo, fecha_nac, estado)
		tel = f'({ct_ciudad["lada"]})' + ' ' + '{:03d}'.format(rm.randint(0, 999)) + ' ' + '{:04d}'.format(rm.randint(0, 9999))
		profesion = profesiones.sample().values[0][0] if edad >= 18 else None

		# Generar un nombre de usuario para un email
		switcher = {
			1: fake.user_name(),
			# aperez
			2: f'{nombre[0]}{apellido_paterno}',
			# alonso_perez
			3: f'{nombre}_{apellido_paterno}',
			# aps70
			4: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.strftime("%y")}',
			# aps1970
			5: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.year}',
			# alonsoperezsoltero
			6: f'{nombre}{apellido_paterno}{apellido_materno}'
		}
		rand_int = rm.randint(min(list(switcher.keys())), max(list(switcher.keys())))
		usuario = switcher.get(rand_int, 'usuario')
		# quitar espacios o reemplazarlos por un _
		usuario = usuario.replace(' ', rm.choice(['', '_'])).lower()
		usuario = quitar_acentos(usuario)
		subdominio = rm.choice(['.com', '.net', '.io', '.gob', '.org', '.edu'])
		correo = usuario + f'@example{subdominio}'

		"""
		El Número de Seguridad Social (NSS) es una homoclave que asigna el IMSS
		para llevar un registro de los trabajadores y asegurados
		que están inscritos en dicha institución.
		El NSS es único, permanente e intransferible.
		"""
		# El NSS está compuesto por 11 dígitos:
		nss = None
		if profesion:
			# Los primeros dos caracteres están vinculados a la subdelegación en el que la persona fue afiliada.
			nss1 = ('%02d' % rm.randint(0, 99))
			# Los dos dígitos siguientes indican el año en el que la persona se afilió al Seguro Social.
			# suponiendo que la persona empezo a trabajar de sus 18 a sus 25
			nss2 = (fecha_nac + pd.DateOffset(months = rm.randint(216, 300))).strftime('%y')
			# Los siguientes dos dígitos corresponden a la fecha de nacimiento del afiliado.
			nss3 = fecha_nac.strftime('%y')
			# Los cuatro números siguientes son los dígitos que asigna el IMSS al trabajador.
			nss4 = ('%04d' % rm.randint(0, 9999))
			# El último dígito corresponde al número de verificación del trabajador en el IMSS.
			nss5 = str(rm.randint(0, 9))
			nss = f'{nss1} {nss2} {nss3} {nss4} {nss5}'

		"""Coordenadas de la ciudad"""
		centro = coordenadas = None
		# si el catalogo ya tiene coordenadas, tomar esos valores
		if pd.notna(ct_ciudad['centro']) and pd.notna(ct_ciudad['suroeste']) and pd.notna(ct_ciudad['noreste']):
			centro = eval(ct_ciudad['centro'])
			coordenadas = {
				'centro': centro,
				'suroeste': eval(ct_ciudad['suroeste']),
				'noreste': eval(ct_ciudad['noreste'])
			}
		else:
			# si no, buscar en internet el punto central y extremos segun la ciudad
			try:
				coordenadas = obtener_coordenadas(ciudad, estado)
				if coordenadas:
					centro = coordenadas['centro']
					# guardar coordenadas en el catalogo
					# para que a la otra no haya que buscar en internet
					ct_ciudades.loc[ct_ciudad_index_row, 'centro'] = str(centro)
					ct_ciudades.loc[ct_ciudad_index_row, 'suroeste'] = str(coordenadas['suroeste'])
					ct_ciudades.loc[ct_ciudad_index_row, 'noreste'] = str(coordenadas['noreste'])
					ct_ciudades.to_csv(file_ciudades, index = False)
			except BaseException as be:
				# En caso de no haber encontrado o no tener internet
				print(f'\033[31mCoordsException: idx {i}\033[0m')
				print(be)

		"""Direccion"""
		direcc = cp = None
		if centro and coordenadas:
			puntos_random = generar_puntos_random(coordenadas, 5)
			try:
				direccion = buscar_direccion(puntos_random)
				direcc = ''
				if direccion and all(k in direccion for k in ('calle', 'numero')):
					direcc = f"{direccion['calle']}, {direccion['numero']}"
				direcc += f", {direccion['colonia']}" if 'colonia' in direccion else ''
				cp = direccion['cp'] if 'cp' in direccion else None
			except BaseException as be:
				print(f'\033[31mAddressException: idx {i}\033[0m')
				print(be)

		"""Zona horaria"""
		timezone = tz_offset = None
		if centro:
			# obtener zona horaria segun latitud y longitud
			obj = TimezoneFinder()
			#if centro[1] > 180.0 or centro[1] < -180.0 or centro[0] > 90.0 or centro[0] < -90.0:
			timezone = obj.timezone_at(lat = centro[0], lng = centro[1])
			# en Enero no hay DST
			#tz_offset = pytz.timezone(timezone).localize(dt.datetime(2011, 1, 1)).strftime('%z')
			# 'datetime.now' tiene DST (Daylight Saving Time)
			tz_offset = dt.datetime.now(pytz.timezone(timezone)).strftime('%z')

		if not centro: centro = (np.nan, np.nan)

		personas.append({
			'titulo': titulo,
			'nombre': nombre,
			'apellido_paterno': apellido_paterno,
			'apellido_materno': apellido_materno,
			'fecha_nac': fecha_nac.strftime('%Y-%m-%d'),
			'sexo': sexo,
			'altura': altura,
			'peso': peso,
			'tipo_sangre': tipo_sangre,
			'curp': curp,
			'correo': correo,
			'tel': tel,
			'color_fav': fake.safe_color_name(),
			'profesion': profesion,
			'nss': nss,
			'direccion': direcc,
			'cp': cp,
			'ciudad': ciudad,
			'estado': estado,
			'lat': centro[0],
			'lon': centro[1],
			'zona_horaria': timezone,
			'tz_offset': tz_offset,
		})
	return pd.DataFrame(personas)

personas_aleatorias(10)

Idx0
{'county': 'Coquimatlán', 'state': 'Colima', 'postcode': '28410', 'country': 'México', 'country_code': 'mx'}
{'village': 'Pueblo Juárez (La Magdalena)', 'county': 'Coquimatlán', 'state': 'Colima', 'postcode': '28410', 'country': 'México', 'country_code': 'mx'}
{'county': 'Coquimatlán', 'state': 'Colima', 'postcode': '28410', 'country': 'México', 'country_code': 'mx'}
{'road': 'Constituyentes Eje Pte. 135', 'village': 'Pueblo Juárez (La Magdalena)', 'county': 'Coquimatlán', 'state': 'Colima', 'postcode': '28410', 'country': 'México', 'country_code': 'mx'}
buscar_direccion(): Direccion encontrada en (19.17303843232916, -103.95373037064577)
 {'calle': 'Constituyentes Eje Pte. 135', 'numero': 'S/N', 'cp': '28410', 'ciudad': 'Pueblo Juárez (La Magdalena)', 'municipio': 'Coquimatlán', 'estado': 'Colima', 'pais': 'México'} 
Idx1
obtener_coordenadas(): No se encontraron coordenadas de "El Compás", "Durango", "Mexico"
Idx2
obtener_coordenadas(): Coordenadas encontradas en "San Buenaventura

,titulo,nombre,apellido_paterno,apellido_materno,fecha_nac,sexo,altura,peso,tipo_sangre,curp,...,profesion,nss,direccion,cp,ciudad,estado,lat,lon,zona_horaria,tz_offset
0,None,Itzel,Salazar,Corrales,2020-05-05,M,75.86,7.77,B+,SACI200505MCMAOTV3,...,None,None,"Constituyentes Eje Pte. 135, S/N",28410,Coquimatlán,Colima,19.175978,-103.923913,America/Mexico_City,-0500
1,None,Conrado,Soto,Fonseca,1909-02-08,H,178.40,62.04,A+,SOFC090208HDGOOO87,...,Voluntario,90 31 09 8239 2,None,None,El Compás,Durango,NaN,NaN,None,None
2,None,Raúl,Espinoza,Colón,1910-10-29,H,180.62,85.92,B-,ESCR101029HCHSOA95,...,Desarrollador de aplicaciones,02 33 10 0131 7,"Calle 17, S/N",31890,San Buenaventura,Chihuahua,29.842766,-107.460250,America/Chihuahua,-0600
3,Dra.,Dalia,Ramón,Guajardo,1949-07-30,M,171.86,51.30,O+,RAGD490730MPLAUA08,...,Redactor de publicidad,49 70 49 3442 9,"Boulevard Cuauhtémoc Norte, 213",75200,Tepeaca,Puebla,18.968056,-97.902500,America/Mexico_City,-0500
4,None,Eva,Santana,Sarabia,1924-03-18,M,170.07,52.79,O+,SASE240318MTSAAV36,...,Administrador de educación,65 42 24 2380 8,"Calle General Ignacio Zaragoza, S/N",None,Guardados de Arriba,Tamaulipas,26.380562,-98.965726,America/Matamoros,-0500
5,Sra.,María del Carmen,Carrasco,Navarrete,1955-01-29,M,154.28,60.74,O+,CANM550129MZSAAA45,...,Curador,18 77 55 8886 3,None,None,Campo Cinco (La Honda),Zacatecas,NaN,NaN,None,None
6,None,Oswaldo,Maldonado,Guzmán,1926-02-19,H,176.21,81.91,A+,MAGO260219HPLAUS38,...,Científico biomédico,15 49 26 7196 0,"Calle 16 de Septiembre Norte, S/N",73680,Zacapoaxtla,Puebla,19.873106,-97.588998,America/Mexico_City,-0500
7,Mtra.,Ofelia,Aguilera,Vega,1984-07-20,M,163.10,65.12,O-,AGVO840720MCCGEF31,...,Porrista,21 07 84 2882 4,,None,Candelaria,Campeche,18.186072,-91.043117,America/Merida,-0500
8,None,María del Carmen,Alfaro,Caraballo,2007-09-21,M,159.81,65.92,A+,ALCM070921MBCLAAI7,...,None,None,"Ramal a Santa María, S/N",22950,Venustiano Carranza,Baja California,30.401797,-115.893732,America/Tijuana,-0700
9,Lic.,Maximiliano,Vergara,Orellana,1936-09-05,H,183.23,76.06,O+,VEOM360905HNLERA05,...,Podólogo,69 58 36 8669 1,"Avenida Protacio Rodríguez Cuéllar, S/N, Centro",67740,Linares,Nuevo Leon,24.859628,-99.568454,America/Monterrey,-0500
